In [ ]:
import guitarpro
import dadagp as dada
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import json

In [ ]:
#path to test file
DADAGP_PATH = "acdc_ex.txt"
GP_PATH = "../util/test_acdc.gp5"
PREFIX = "./test_loops_out/acdc_test"

In [ ]:
#snippet for extracting time signatures

song = guitarpro.parse(GP_PATH) #check if one time signature throughout, calculate min/max beat lengths based on it
timeSignatures = {}
for measure in song.tracks[0].measures[0:10]:
    signature = (measure.timeSignature.numerator, measure.timeSignature.denominator.value)
    if signature in timeSignatures.keys():
        timeSignatures[signature] += 1
    else:
        timeSignatures[signature] = 1
    print(measure.timeSignature.numerator, measure.timeSignature.denominator.value)

In [ ]:
#create the json file for names of all the loop files

root_dir = "D:\Documents\DATA\DadaGP-Loops"
file_list = []
for subdir, dirs, files in os.walk(root_dir):
    short_subdir = subdir.replace(root_dir + "\\", "")
    for file in files:
        if file.endswith(".txt"):
            full_path = os.path.join(short_subdir, file)
            file_list.append(full_path)

#save that file list to json
root_dir = "D:\Documents\DATA\DadaGP-Loops"
path_json = os.path.join(root_dir, "_DadaGP_LOOPS_all_filenames.json")
with open(path_json, 'w') as f:
    json.dump(file_list, f)

In [ ]:
#helper function for finding number of empty bars in a track

def calc_empty_tracks(test_file):
    song = guitarpro.parse(test_file)
    empty_tracks = 0
    total_tracks = len(song.tracks)
    for track in song.tracks:
        empty_bars = 0
        total_bars = len(track.measures)
        for measure in track.measures:
            non_rests = 0
            for beat in measure.voices[0].beats:
                for note in beat.notes:
                    if note.type != guitarpro.NoteType.rest:
                        non_rests = non_rests + 1
            if non_rests == 0:
                empty_bars = empty_bars + 1
        if empty_bars == total_bars:
            empty_tracks = empty_tracks + 1
    return empty_tracks, total_tracks

In [ ]:
# report the number of empty bars across all tracks
import json

root_path = "D:\Documents\DATA\DadaGP-4-8-lps-3-dens-per-inst-hard-reps"
allfiles_path = os.path.join(root_path,"file_list_loops.json" )
with open(allfiles_path, "r") as f:
    allfiles =  json.load(f)

n_files = len(allfiles)
total_tracks = 0
empty_tracks = 0
for fidx in range(n_files):   
    fn = allfiles[fidx].replace(".txt", ".gp5")
    file = os.path.join(root_path, fn)
    empty, total = calc_empty_tracks(file)
    total_tracks = total_tracks + total
    empty_tracks = empty_tracks + empty

print("{} empty tracks out of {} total tracks".format(empty_tracks, total_tracks))

In [ ]:
35003.0 / 176703.0 #percentage of empty tracks in original loop dataset

In [ ]:
#get max number of tokens in dataset and average

root_path = "D:\Documents\DATA\DadaGP-v1.1"
allfiles_path = os.path.join(root_path,"_DadaGP_all_filenames.json" )
with open(allfiles_path, "r") as f:
    allfiles =  json.load(f)

n_files = len(allfiles)
max_lines = 0
total_lines = 0
max_song = None
for fidx in range(n_files):   
    fn = allfiles[fidx]
    file = os.path.join(root_path, fn)
    try:
        with open(file) as f:
            num_lines = len(f.readlines())
            total_lines += num_lines
            if num_lines > max_lines:
                max_lines = num_lines
                max_song = fn
    except:
        print("skipping ", file)
        continue
print("{} maximum token length from {}".format(max_lines, max_song))
print("{} average tokens per song".format(total_lines * 1.0 /n_files))

In [ ]:
#average tracks per loop

root_path = "D:\Documents\DATA\DadaGP-Loops"
allfiles_path = os.path.join(root_path,"file_list.json" )
with open(allfiles_path, "r") as f:
    allfiles =  json.load(f)

n_files = len(allfiles)
total_tracks = 0
for fidx in range(n_files):   
    fn = allfiles[fidx].replace(".txt", ".gp5")
    file = os.path.join(root_path, fn)
    song = guitarpro.parse(file)
    total_tracks = total_tracks + len(song.tracks)
print("{} average tracks per file".format(total_tracks * 1.0 / n_files))

In [ ]:
#print out all the repeats in a test song
test_file = "D:\\Documents\\DATA\\DadaGP-v1.1\\1\\1349\\1349 - Deathmarch.gp4"
song = guitarpro.parse(test_file)
for i, measure in enumerate(song.tracks[0].measures):
    if measure.header.isRepeatOpen:
        print("repeat open at {}".format(i))
    elif measure.header.repeatClose > -1:
        print("repeat close {}x at {}".format(measure.header.repeatClose, i))
#isRepeatOpen will be true at the beginning of the repeat. repeatClose is an int that appears in the measure of the closing repeat
#number repeats does not include first playthrough


In [ ]:
#analyze test file with alternate endings

test_file = "D:\\Documents\\DATA\\DadaGP-v1.1\\3\\3 Inches Of Blood\\3 Inches Of Blood - Balls Of Ice.gp4"
song = guitarpro.parse(test_file)
for i, measure in enumerate(song.tracks[0].measures):
    if measure.header.isRepeatOpen:
        print("repeat open at {}".format(i))
    elif measure.header.repeatClose > -1:
        print("repeat close {}x at {}".format(measure.header.repeatClose, i))
    elif measure.header.repeatAlternative > 0:
        print("repeat alt {} at {}".format(measure.header.repeatAlternative, i))

In [ ]:
#code for reformatting the data parallel model
import torch
from collections import OrderedDict

# original saved file with DataParallel
model = torch.load('../outputs/5_lat102420220714-235039/parallel_ep_40.pth.tar')

# create new OrderedDict that does not contain `module.`
new_state_dict = OrderedDict()
for k, v in model['state_dict'].items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
    
# load params
model['state_dict'] = new_state_dict
torch.save(model, '../outputs/5_lat102420220714-235039/ep_40.pth.tar')